In [2]:
import numpy as np
import pandas as pd
from obspy import read
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.signal import spectrogram
from scipy.ndimage import label
from scipy.ndimage import gaussian_filter1d
import os
import keras
from keras import layers

In [3]:
class SeismicDataProcessor:
    def __init__(self, csv_file, mseed_file):
        """
        Initializes the processor with the provided CSV and MiniSEED files.
        Args:
            csv_file (str): Path to the CSV file with time and velocity data.
            mseed_file (str): Path to the MiniSEED file with frequency data.
        """
        self.csv_file = csv_file
        self.mseed_file = mseed_file
        self.load_data()

    def load_data(self):
        """Loads CSV and MiniSEED data."""
        try:
            # Load CSV data for time and velocity
            self.csv_data = pd.read_csv(self.csv_file)
            if 'time_rel(sec)' in self.csv_data.columns:
                self.csv_data.rename(columns={'time_rel(sec)': 'rel_time(sec)'}, inplace=True)

            if 'velocity(m/s)' in self.csv_data.columns:
                self.csv_data.rename(columns={'velocity(m/s)': 'velocity(c/s)'}, inplace=True)

            if 'time_abs(%Y-%m-%dT%H:%M:%S.%f)' in self.csv_data.columns:
                self.csv_data.rename(columns={'time_abs(%Y-%m-%dT%H:%M:%S.%f)': 'time(%Y-%m-%dT%H:%M:%S.%f)'}, inplace=True)
            
            print(self.csv_data.head())

            self.time = self.csv_data['rel_time(sec)'].values
            self.velocity = self.csv_data['velocity(c/s)'].values

            # Load MiniSEED data for spectrogram
            self.mseed_data = read(self.mseed_file)
            trace = self.mseed_data[0]  # Assuming single trace
            self.sampling_rate = trace.stats.sampling_rate
            self.sxx = None
            self.frequencies = None
            self.calculate_spectrogram(trace.data)

        except Exception as e:
            print(f"Error loading data: {e}")

    def calculate_spectrogram(self, velocity):
        """Calculates the spectrogram from the velocity data."""
        self.frequencies, self.time_spec, self.sxx = spectrogram(velocity, fs=self.sampling_rate)

    def calculate_power(self):
        """
        Calculate the power of the velocity signal from the spectrogram data.
        Returns:
            power (ndarray): The power of the signal over time.
        """
        # Power is the square of the spectrogram (sxx) values
        power = np.abs(self.sxx) ** 2
        return np.mean(power, axis=0)  # Average across frequencies

    def detect_power_clusters(self, power_scaled, percentile=90):
        """Detects clusters in the power data based on a threshold."""
        power_threshold = np.percentile(power_scaled, percentile)
        power_above_threshold = power_scaled > power_threshold
        labeled_clusters, num_clusters = label(power_above_threshold)

        # Identify the cluster with the maximum total power
        max_power_cluster = None
        max_cluster_sum = 0

        for cluster_id in range(1, num_clusters + 1):
            cluster_indices = np.where(labeled_clusters == cluster_id)[0]
            cluster_sum = np.sum(power_scaled[cluster_indices])

            if cluster_sum > max_cluster_sum:
                max_cluster_sum = cluster_sum
                max_power_cluster = cluster_indices

        arrival_time = self.time_spec[max_power_cluster[0]] if max_power_cluster is not None else None
        return max_power_cluster, arrival_time

    def smooth_power(self, power, window_len=10):
        """Smooths the power data using a simple moving average."""
        return np.convolve(power, np.ones(window_len) / window_len, mode='same')

    def plot_results(self, power_scaled, max_power_cluster, arrival_time):
        """Plots the time series, spectrogram, and power clusters."""
        plt.figure(figsize=(10, 10))
        self.plot_velocity_time_series(arrival_time)
        self.plot_spectrogram(arrival_time)
        self.plot_power_and_clusters(power_scaled, max_power_cluster, arrival_time)

    def plot_velocity_time_series(self, arrival_time):
        """Plots the velocity time series with the detected trigger time."""
        plt.subplot(3, 1, 1)
        plt.plot(self.time, self.velocity, label="Velocity (c/s)")
        plt.xlabel('Time (sec)')
        plt.ylabel('Velocity (c/s)')
        plt.title('Velocity Time Series')
        if arrival_time:
            plt.axvline(arrival_time, color='r', linestyle='--', label='Trigger Time')
        plt.legend()
        plt.savefig('images/velocity_time_series.png')
        plt.close() 

    def plot_spectrogram(self, arrival_time):
        """Plots the spectrogram of the velocity data."""
        plt.subplot(3, 1, 2)
        plt.pcolormesh(self.time_spec, self.frequencies, 10 * np.log10(self.sxx), shading='gouraud')
        plt.xlabel('Time (sec)')
        plt.ylabel('Frequency (Hz)')
        plt.title('Spectrogram (dB)')
        if arrival_time:
            plt.axvline(arrival_time, color='r', linestyle='--', label='Trigger Time')
        plt.colorbar()
        plt.savefig('images/spectrogram.png')
        plt.close() 

    def plot_power_and_clusters(self, power_scaled, max_power_cluster, arrival_time):
        """Plots the scaled power and detected clusters."""
        plt.subplot(3, 1, 3)
        plt.plot(self.time_spec, power_scaled, label='Smoothed Power', color='b')
        if max_power_cluster is not None:
            plt.scatter(self.time_spec[max_power_cluster], power_scaled[max_power_cluster], color='r', label='Detected Cluster')
        plt.xlabel('Time (sec)')
        plt.ylabel('Power')
        plt.title('Power Over Time')
        if arrival_time:
            plt.axvline(arrival_time, color='r', linestyle='--', label='Trigger Time')
        plt.legend()
        plt.savefig('images/power_clusters.png')
        plt.close()

    def process_data(self):
        """Main processing pipeline to load data, compute spectrogram, detect clusters, and plot results."""
        power_scaled = self.calculate_power()
        smoothed_power = self.smooth_power(power_scaled)

        # Detect power clusters and arrival time
        max_power_cluster, arrival_time = self.detect_power_clusters(smoothed_power)

        # Plot the results
        self.plot_results(smoothed_power, max_power_cluster, arrival_time)

        return arrival_time

In [4]:
# Usage Example
if __name__ == "__main__":
    csv_files = []
    mseed_files = []
   
    trigger_data = pd.DataFrame(columns=['filename', 'detected_trigger_time','timestamp'])
    
    csv_files=["data/mars/test/data/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.csv"]
    mseed_files=["data/mars/test/data/XB.ELYSE.02.BHV.2019-05-23HR02_evid0041.mseed"]

    # Process each file pair
    for csv_file, mseed_file in zip(csv_files, mseed_files):
        processor = SeismicDataProcessor(csv_file,mseed_file)
        arrival_time = processor.process_data()
        closest_index = (np.abs(processor.time - arrival_time)).argmin()
        corresponding_timestamp = processor.csv_data['time(%Y-%m-%dT%H:%M:%S.%f)'].iloc[closest_index]

        # Add the result to the trigger data DataFrame using pd.concat()
        new_row = pd.DataFrame({
            'filename': [os.path.basename(csv_file)],
            'detected_trigger_time': [arrival_time],
            'timestamp': [corresponding_timestamp]
        })

        trigger_data = pd.concat([trigger_data, new_row], ignore_index=True)

    # Save the detected trigger times to a CSV file
    trigger_data.to_csv('final.csv', index=False)

   time(%Y-%m-%dT%H:%M:%S.%f)  rel_time(sec)  velocity(c/s)
0  2019-05-23T02:00:00.032000           0.00      -0.000000
1  2019-05-23T02:00:00.082000           0.05       0.000199
2  2019-05-23T02:00:00.132000           0.10      -0.001630
3  2019-05-23T02:00:00.182000           0.15      -0.000875
4  2019-05-23T02:00:00.232000           0.20      -0.006137


/var/folders/0y/_qpkhqy57dd3pfbf16f5k2dm0000gn/T/ipykernel_14066/1450868812.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  trigger_data = pd.concat([trigger_data, new_row], ignore_index=True)
